In [1]:
import docx
#import io
import pandas as pd
import os


In [3]:
def parse_transcript(file_path,file,subdir):
    data = pd.DataFrame(columns=['text'])
    for para in docx.Document(file_path).paragraphs:
        data.loc[len(data)] = {'text': para.text}
    data = data.text.str.split(':',expand=True)
    data = data.rename(columns={0:'heading'})
    data['transcript'] =data[data.columns[1:]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    data['transcript'] = data['transcript'].str.replace('None','',regex=True)
    data = data[data['transcript']!='']
    data = data[['heading','transcript']]
    data['heading'] = data['heading'].str.strip()
    data['transcript'] = data['transcript'].str.strip()
    for metadata in ['Date','Durée','Lieu','Participant','Etat civil']:
        try: 
            data[metadata] = data[data['heading']==metadata]['transcript'].astype(str).values[0]
        except:
            data[metadata] = None
    data['file'] = file
    data['directory'] = subdir
    return data

In [7]:
directory = input("Enter the full path of the director with all the files: ")

In [8]:
data = pd.DataFrame()
for subdir, dirs, files in os.walk(directory):
    for file in files:
        try: 
            data = pd.concat([data, parse_transcript(os.path.join(subdir, file),file,subdir.replace(directory,''))])
        except:
            print(os.path.join(subdir, file))

C:\Users\jason\Downloads\Transcriptions IEA RDC\Transcriptions IEA RDC\FDG N-K\~$G NYIRAGONGO.docx
C:\Users\jason\Downloads\Transcriptions IEA RDC\Transcriptions IEA RDC\KII N-K\IMG_20220512_202356_resized_20220512_083246882.jpg
C:\Users\jason\Downloads\Transcriptions IEA RDC\Transcriptions IEA RDC\KII N-K\IMG_20220512_202703_resized_20220512_083246604.jpg


In [5]:
data.to_excel(directory+'transcript_data.xlsx')